In [ ]:
!pip install pyspark
!pip install -q google-cloud-storage==1.43.0
from inverted_index_gcp import *
import pyspark


In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 30

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except:
   pass

In [ ]:
# inverted text upload
bucket_name = "bucket324955145"
file_path = "postings_gcp/text/postings_gcp_text_inverted_text_v1.pkl"
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_path)
contents = blob.download_as_bytes()
inverted_text = pickle.loads(contents)

In [ ]:
# inverted title upload
bucket_name = "bucket324955145"
file_path = "postings_gcp/title/postings_gcp_title_inverted_title_v1.pkl"
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_path)
contents = blob.download_as_bytes()
inverted_title = pickle.loads(contents)

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

In [ ]:
# doc len
text_doc_len = inverted_text.doc_len.copy()
title_doc_len = inverted_title.doc_len.copy()

In [ ]:
# idf
text_idf = inverted_text.idf
title_idf = inverted_title.idf

In [ ]:
# idf bm 25
# text_idf_bm25 = inverted_text.get_idf_bm25()
# title_idf_bm25 = inverted_title.get_idf_bm25()
text_idf_bm25 = inverted_text.idf_bm25
title_idf_bm25 = inverted_title.idf_bm25

In [ ]:
# advl
# text_avdl = inverted_text.get_avdl()
# title_avdl = inverted_title.get_avdl()
text_avdl = inverted_text.avdl
title_avdl = inverted_title.avdl

In [ ]:
# check to see what values we can work with now:
print(len(inverted_text.doc_len.copy()))
print(len(inverted_title.doc_len.copy()))
print(len(inverted_text.idf))
print(len(inverted_title.idf))
print(len(text_idf_bm25))
print(len(title_idf_bm25))
print(text_avdl)
print(title_avdl)

6347138
6340943
495532
103028
495532
103028
319.6134407350211
2.5288798527285294


In [ ]:
inverted_title.doc_len[]

In [ ]:
k1 = 0.1
k3 = 0
b = 0.75

In [ ]:
# testing:


def retrieve_posting_list_title(query_word: str, bucket_name: str):
    return inverted_title.read_a_posting_list(base_dir='.', w=query_word, bucket_name=bucket_name)

def retrieve_posting_list_text(query_word: str, bucket_name: str):
    return inverted_text.read_a_posting_list(base_dir='.', w=query_word, bucket_name=bucket_name)

def calculate_bm25_per_tf(token, doc_id_tf, inverted):
    doc_id, tf = doc_id_tf
    B = 1 - b + b * inverted_title.doc_len.get(doc_id)
    return ((k1 + 1) / (B * k1 + tf)) * ((inverted_title.corpus_size + 1) /inverted_title.idf_bm25[token])


# def calculate_bm25(tokens_pl):
#     for token_pl in tokens_pl.items():
#         calculate_bm25_per_token(token_pl)




def query(query: str, bucket_name: str):
    tokens = [token.group() for token in RE_WORD.finditer(query.lower())]
    tokens = [token for token in tokens if token not in all_stopwords]
    token_pl_title = {token: retrieve_posting_list_title(token) for token in tokens}
    bm25_title = calculate_bm25(token_pl_title)

In [ ]:
print(calculate_bm25_per_tf(token='anarchism', doc_id_tf=(12, 1), inverted=inverted_text))
print(calculate_bm25_per_tf(token='anarchism', doc_id_tf=(12, 1), inverted=inverted_text))

405323.9668576603
405323.9668576603


In [ ]:
pls = [
    [(1, 3), (3, 1)],
    [(1, 2), (2, 2), (4, 1)],
    [(2, 3), (3, 3)]
]

In [ ]:
gens = [(item for item in pl) for pl in pls]